In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

from xgboost import XGBRegressor


## Predict rent prices
1. Use pipeline
2. Cross validation
3. XGBoost

In [3]:
raw_data = pd.read_csv("../data/House_Rent_Dataset.csv")
print(raw_data.shape)
raw_data.head(5)

(4746, 12)


,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [4]:
# Drop unnecessary columns, and seperate the target column
drop_col = ['Posted On', 'Rent', 'Tenant Preferred', 'Point of Contact', 'Floor', 'Area Locality']
target = raw_data['Rent']
raw_data_aug = raw_data.drop(columns=drop_col)
print(f"raw_data_aug shape:", raw_data_aug.shape)
print(raw_data_aug.info())
raw_data_aug.head()

raw_data_aug shape: (4746, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   BHK                4746 non-null   int64 
 1   Size               4746 non-null   int64 
 2   Area Type          4746 non-null   object
 3   City               4746 non-null   object
 4   Furnishing Status  4746 non-null   object
 5   Bathroom           4746 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 222.6+ KB
None


,BHK,Size,Area Type,City,Furnishing Status,Bathroom
0,2,1100,Super Area,Kolkata,Unfurnished,2
1,2,800,Super Area,Kolkata,Semi-Furnished,1
2,2,1000,Super Area,Kolkata,Semi-Furnished,1
3,2,800,Super Area,Kolkata,Unfurnished,1
4,2,850,Carpet Area,Kolkata,Unfurnished,1


In [5]:
# Split raw data into train and test
X_train, X_test, y_train, y_test = train_test_split(raw_data_aug, target, test_size=0.25)
print(f"Train split: ",X_train.shape, y_train.shape)
print(f"Test split: ",X_test.shape, y_test.shape)

Train split:  (3559, 6) (3559,)
Test split:  (1187, 6) (1187,)


In [6]:
# Split prior train into train and validation
x_train, x_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)
print(f"Train split: ",x_train.shape, y_train.shape)
print(f"Validation split: ",x_valid.shape, y_valid.shape)

Train split:  (2847, 6) (2847,)
Validation split:  (712, 6) (712,)


## Use Pipeline

In [7]:
num_cols = [cname for cname in x_train.columns if x_train[cname].dtype in ['int32', 'int64']]
cat_cols = [cname for cname in x_train.columns if x_train[cname].dtype=='object']
print(num_cols)
print(cat_cols)

['BHK', 'Size', 'Bathroom']
['Area Type', 'City', 'Furnishing Status']


In [8]:
numerical_transform = SimpleImputer(strategy="most_frequent")
categorical_transform = Pipeline(steps=[
    ('impute', SimpleImputer(strategy="most_frequent")),
    ('encode', OneHotEncoder(sparse=False, handle_unknown='ignore')) # LabelEncoder doesnt work, why sparse=False?
])

preprocessing = ColumnTransformer(transformers=[
    ('num',numerical_transform,num_cols),
    ('cat', categorical_transform,cat_cols)
])

In [9]:
model = RandomForestRegressor()

In [10]:
my_pipeline = Pipeline(steps=[
    ('preprocess', preprocessing),
    ('model', model)
])
my_pipeline

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['BHK', 'Size', 'Bathroom']),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['Area Type', 'City',
                                                   'Furnishing Status'])])),
                ('model', RandomForestRegressor())])

In [ ]:
my_pipeline = my_pipeline.fit(x_train, y_train)
preds = my_pipeline.predict(x_valid)

In [ ]:
# Compare preds to actual
print("Valid MAE:", mean_absolute_error(y_valid, preds))

In [ ]:
# Generate predictions on test set
test_preds = my_pipeline.predict(X_test)
print("Test MAE:", mean_absolute_error(y_test, test_preds))
pd.DataFrame(test_preds)

## Cross Validation

In [ ]:
def get_score(n_estimators):
    model = RandomForestRegressor(n_estimators, random_state=1)
    my_pipeline = Pipeline(steps=[
        ('preprocess', preprocessing),
        ('model', model)
    ])
    
    scores = -1 * cross_val_score(my_pipeline, x_train, y_train, cv=5, scoring='neg_mean_absolute_error')
    return scores.mean()

In [ ]:
get_score(50)

In [ ]:
results = {}
for i in range(1, 9):
    results[i*50] = get_score(i*50)

In [ ]:
results

In [ ]:
plt.plot(list(results.keys()), list(results.values()))
plt.xlabel('n_estimator')
plt.ylabel('mean score')
plt.show()

## XG Boost

In [ ]:
model = XGBRegressor()

my_pipeline = Pipeline(steps=[
    ('preprocess', preprocessing),
    ('model', model)
])

In [ ]:
my_pipeline.fit(x_train, y_train)

In [ ]:
predictions = my_pipeline.predict(x_valid)

In [ ]:
mae = mean_absolute_error(y_valid, predictions)
mae